In [1]:
import os
from nipype.interfaces import afni as afni
import nipype.pipeline.engine as pe 
import nipype.interfaces.io as nio
import nipype.interfaces.utility as util

import warnings
import re
import sys
from nipype.interfaces.afni.base import AFNICommand, AFNICommandInputSpec
from nipype.interfaces.base import CommandLineInputSpec, CommandLine, TraitedSpec, traits, File, InputMultiPath, \
    isdefined
import glob

#from nipype.interfaces.afni.ccpafni import CCPDeconvolve
from interfaces import CCPDeconvolve, OneDTool, CCPMaskTool
data_dir = os.path.abspath('../DATA')
print data_dir
print os.listdir(data_dir)

/home/brain/nipype/ccp/DATA
['sub003', 'sub001', 'union', 'ccpdeconvolve', 'mean', 'tcat', 'movdemean', 'scale', 'wf', 'outputs_WF1', 'voxeldim', 'outputs_AFNI']


In [2]:
# DATA
subject_list = ['sub001', 'sub003'] 
runs = ['run001', 'run002']
#niftis = glob.glob(os.path.abspath(data_dir+'/sub001/run*'))
#niftis.sort()
#niftis
# movregs = os.path.abspath('../DATA/Movement_Regressors_ALL.txt')
tr_counts = [1000, 1000, 1000, 761]

In [3]:
infosource = pe.Node(util.IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

In [4]:
# NODE SELECT_FILES

templates = {'myfiles': data_dir + "/{subject_id}/run00[1,2].nii.gz",
             'movregs': data_dir + "/{subject_id}/Movement_Regressors_ALL.txt",
             'onsets': data_dir + "/{subject_id}/onsets_COND00[1,3]_ALL.txt"}

sf = pe.Node(interface=nio.SelectFiles(templates), 
             name='sf')
#sf.iterables = ('subject_id', subject_list)
sf.inputs.base_directory = data_dir
sf.inputs.sort_filelist = True
#sf.inputs.force_lists = True
#sf.inputs.subject_id = 'sub001'
sf.inputs.force_lists = ['myfiles', 'onsets']
#sf.inputs.subject_id = 'subject_id'
sf.inputs




base_directory = /home/brain/nipype/ccp/DATA
force_lists = ['myfiles', 'onsets']
ignore_exception = False
raise_on_empty = True
sort_filelist = True
subject_id = <undefined>

In [5]:
#sf.run()

In [6]:
# NODE
#1d_tool.py -infile $movregfile         	                         \
#           -set_run_lengths $tr_counts	                         \
#           -derivative  -collapse_cols euclidean_norm            \
#           -write motion_${subj}_enorm.1D

wf_mov_magnitude = pe.Node(interface=OneDTool(),
                 name='wf_mov_magnitude')

wf_mov_magnitude.base_dir = data_dir
#wf_mov_magnitude.inputs.in_file = movregs
wf_mov_magnitude.inputs.tr_counts = tr_counts
wf_mov_magnitude.inputs.out_file = 'mov_magnitude_enorm.1D'
wf_mov_magnitude.inputs.args = '-derivative'
wf_mov_magnitude.inputs.collapse_cols = 'euclidean_norm'
wf_mov_magnitude.inputs


args = -derivative
collapse_cols = euclidean_norm
environ = {}
ignore_exception = False
in_file = <undefined>
out_file = mov_magnitude_enorm.1D
terminal_output = stream
tr_counts = [1000, 1000, 1000, 761]

In [7]:
wf1 = pe.Workflow(name="wf1")
wf1.base_dir = data_dir
#my_workflow.connect(sf, 'myfiles', to_afni, 'in_file')
#my_workflow.connect(to_afni, 'out_file', tcat, 'in_files')
wf1.connect(infosource, 'subject_id', sf, 'subject_id')
#wf1.add_nodes([sf, wf_mov_magnitude])
wf1.connect([(sf, wf_mov_magnitude,[('movregs', 'in_file')])
             ]) 

In [8]:
# NODE AUTOMASK
wf_automask = pe.MapNode(interface=afni.Automask(),
                   name="wf_automask",
                     iterfield=['in_file'])
#wf_automask.inputs.in_file = niftis
wf_automask.inputs.dilate = 1
#wf_automask.base_dir = data_dir
#wf_automask.inputs.out_file = 'mask'
#wf_automask.inputs.brain_file = 'data_masked_run'
#wf_automask.base_dir = data_dir
wf_automask.inputs.outputtype = 'NIFTI_GZ'
#wf_automask.inputs

In [9]:
wf_mask_union = pe.Node(interface=CCPMaskTool(),
                       name='wf_mask_union')

In [10]:
#wf_mask_union.base_dir = data_dir
wf_mask_union.inputs.out_file = 'union'
#wf_mask_union.inputs.in_file = glob.glob(os.path.abspath('../DATA/mask*'))
#wf_mask_union.inputs.in_file.sort()ls
wf_mask_union.inputs.args = "-union"
wf_mask_union.inputs.outputtype = 'NIFTI_GZ'
#wf_mask_union.inputs

In [11]:
#NODE MEAN
wf_mean = pe.MapNode(interface=afni.TStat(),
                     name='wf_mean',
                     iterfield=['in_file'])
#mean.base_dir = data_dir
wf_mean.inputs.out_file = 'mean'
#wf_mean.inputs.in_file = os.path.abspath('../DATA/run001+orig.BRIK')
#wf_mean.inputs


In [12]:

#my_workflow.connect(sf, 'myfiles', to_afni, 'in_file')
#my_workflow.connect(to_afni, 'out_file', tcat, 'in_files')
wf1.add_nodes([wf_automask, wf_mask_union, wf_mean])
wf1.connect([(sf, wf_automask,[('myfiles', 'in_file')]),
             (wf_automask, wf_mask_union,[('out_file','in_file')]),
             (sf, wf_mean,[('myfiles','in_file')]),
            ])

In [13]:
# NODE SCALE
wf_scale = pe.MapNode(interface=afni.Calc(),
                      name='wf_scale',
                      iterfield=['in_file_a', 'in_file_b'])
#scale.base_dir = data_dir
wf_scale.inputs.out_file = 'scale'
#scale.inputs.in_file_a = os.path.abspath('../DATA/run001+orig.BRIK')
#scale.inputs.in_file_b = os.path.abspath('../DATA/mean+orig.BRIK')
wf_scale.inputs.expr = 'min(200, a/b*100)*step(a)*step(b)'
wf_scale.inputs.outputtype = 'NIFTI_GZ'
#scale.inputs

In [14]:
wf_scale.inputs

Exception occurred in traits notification handler for object: 
args = <undefined>
environ = {}
expr = min(200, a/b*100)*step(a)*step(b)
ignore_exception = False
in_file_a = <undefined>
in_file_b = <undefined>
in_file_c = <undefined>
other = <undefined>
out_file = scale
outputtype = NIFTI_GZ
single_idx = <undefined>
start_idx = <undefined>
stop_idx = <undefined>
terminal_output = stream
, trait: trait_added, old value: <undefined>, new value: _ipython_display_
Traceback (most recent call last):
  File "/home/brain/anaconda/lib/python2.7/site-packages/traits/trait_notifiers.py", line 340, in __call__
    self.handler( *args )
  File "/home/brain/anaconda/lib/python2.7/site-packages/traits/has_traits.py", line 1072, in _trait_added_changed
    if (trait.type == 'delegate') and (name[-6:] != '_items'):
AttributeError: 'NoneType' object has no attribute 'type'
ERROR:traits:Exception occurred in traits notification handler for object: 
args = <undefined>
environ = {}
expr = min(200, a/b*100)

Exception occurred in traits notification handler.
Please check the log file for details.



args = <undefined>
environ = {}
expr = min(200, a/b*100)*step(a)*step(b)
ignore_exception = False
in_file_a = <undefined>
in_file_b = <undefined>
in_file_c = <undefined>
other = <undefined>
out_file = scale
outputtype = NIFTI_GZ
single_idx = <undefined>
start_idx = <undefined>
stop_idx = <undefined>
terminal_output = stream

In [15]:
wf1.add_nodes([wf_scale])
wf1.connect([(sf, wf_scale,[('myfiles', 'in_file_a')]),
             (wf_mean,wf_scale,[('out_file', 'in_file_b')]),
            ])

In [16]:
# NODE MOTION REGRESSORS (MapNode could work for iterfields = args, out_file)
wf_movdemean = pe.Node(interface=OneDTool(),
                 name='wf_movdemean')
wf_movdemean.base_dir = data_dir
#wf_movdemean.inputs.in_file = movregs
wf_movdemean.inputs.tr_counts = tr_counts
wf_movdemean.inputs.out_file = 'motion_demean.1D'
wf_movdemean.inputs.args = '-demean'

In [17]:
wf_movderiv = pe.Node(interface=OneDTool(),
                  name="wf_movderiv")
#wf_movderiv.inputs.in_file = movregs
wf_movderiv.inputs.tr_counts = tr_counts
wf_movderiv.inputs.out_file = 'motion_deriv.1D'
wf_movderiv.inputs.args = '-derivative -demean'


In [18]:
wf1.add_nodes([wf_movdemean, wf_movderiv])
wf1.connect([(sf, wf_movdemean,[('movregs', 'in_file')]),
             (sf,wf_movderiv,[('movregs', 'in_file')]),
            ])

In [19]:
# NODE CCPDECONVOLVE

In [20]:
# Inputs: scaled images, motion regressors, onset files, 

wf_glm = pe.Node(interface=CCPDeconvolve(),
              name="wf_glm")
#wf_glm.inputs

In [21]:
#glm.base_dir = data_dir
#glm.inputs.in_file = glob.glob(os.path.abspath('../DATA/sub001/run*'))
wf_glm.inputs.polort = 5
wf_glm.inputs.out_file = 'X.xmat.1D'
wf_glm.inputs.img_file = 'X.png'
#wf_glm.inputs.xmat_only = True
wf_glm.inputs.args = '-GOFORIT 10'
#glm.inputs.output_prefix = 'stats'
wf_glm.inputs.num_stimts = 2
wf_glm.inputs.stimulus = True
#wf_glm.inputs.stim_times=glob.glob(os.path.abspath('../DATA/sub001/onsets_*_ALL.txt'))
wf_glm.inputs.stim_labels=['C1', 'C3']
#glm.inputs.glm_contrasts = ['C1 -C3', 'C3 -C1', '0.5*C1 +0.5*C3']
#glm.inputs.glm_labels = ['diff_pos', 'diff_neg', 'mean']
#glm.inputs.glms = True
wf_glm.inputs.options = '-x1D_stop -local_times'
#glm.inputs.args = '-xjpeg X.png'
wf_glm.inputs.model = 'CSPLIN(0,4000,4)'

In [22]:
wf_glm.inputs


args = -GOFORIT 10
censor_list = <undefined>
environ = {}
glm_contrasts = <undefined>
glm_labels = <undefined>
glms = <undefined>
ignore_exception = False
img_file = X.png
in_file = <undefined>
mask = <undefined>
model = CSPLIN(0,4000,4)
motion_regs = <undefined>
num_stimts = 2
options = -x1D_stop -local_times
out_file = X.xmat.1D
output_prefix = <undefined>
outputtype = AFNI
polort = 5
stim_labels = ['C1', 'C3']
stim_times = <undefined>
stimulus = True
terminal_output = stream
xmat_only = <undefined>

In [23]:
wf1.connect([(sf, wf_glm,[('onsets', 'stim_times')]),
             (sf, wf_glm,[('movregs', 'motion_regs')]),
             (wf_scale,wf_glm,[('out_file', 'in_file')]),
            ])

In [24]:
#DATA SINK

In [25]:
#DATA SINK
ds = pe.Node(interface=nio.DataSink(),
            name="ds")
ds.inputs


_outputs = {}
base_directory = <undefined>
container = <undefined>
ignore_exception = False
parameterization = True
regexp_substitutions = <undefined>
remove_dest_dir = False
strip_dir = <undefined>
substitutions = <undefined>

In [26]:
ds.inputs.base_directory = data_dir + '/outputs_WF1'
substitutions = [('_subject_id_', '')]
ds.inputs.substitutions = substitutions
ds.inputs.parameterization = False


In [27]:
wf1.connect(infosource, 'subject_id', ds, 'container')
wf1.connect(sf, 'movregs', ds, 'motion')


In [28]:
wf1.connect(wf_movdemean, 'out_file', ds, 'motion.@demean')
wf1.connect(wf_movderiv, 'out_file', ds, 'motion.@deriv')


In [29]:
#wf.connect(infosource, 'subject_id', sf, 'subject_id')
#wf.connect(infosource, 'subject_id', ds, 'container')
#wf.connect(sf, 'movregs', wf_mov_magnitude, 'in_file')
#wf1.connect(wf_mov_magnitude, 'out_file', ds, 'magnitude')

wf1.connect(sf, 'onsets', ds, 'onsets')

In [30]:
if __name__ == '__main__':
    wf1.run()
    results = wf1.write_graph(graph2use='exec')

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node sf.a0 in dir: /home/brain/nipype/ccp/DATA/wf1/_subject_id_sub001/sf
INFO:workflow:Executing node wf_mean.a0 in dir: /home/brain/nipype/ccp/DATA/wf1/_subject_id_sub001/wf_mean
INFO:workflow:Executing node _wf_mean0 in dir: /home/brain/nipype/ccp/DATA/wf1/_subject_id_sub001/wf_mean/mapflow/_wf_mean0
INFO:workflow:Running: 3dTstat -prefix mean /home/brain/nipype/ccp/DATA/wf1/_subject_id_sub001/wf_mean/mapflow/_wf_mean0/run001.nii.gz
INFO:interface:stderr 2016-02-16T16:11:53.669166:++ 3dTstat: AFNI version=AFNI_2011_12_21_1014 (Dec 27 2014) [64-bit]
INFO:interface:stderr 2016-02-16T16:11:53.671240:++ Authored by: KR Hammett & RW Cox
INFO:interface:stderr 2016-02-16T16:11:55.034037:++ Output dataset ./mean+orig.BRIK
INFO:workflow:Executing node _wf_mean1 in dir: /home/brain/nipype/ccp/DATA/wf1/_subject_id_sub001/wf_mean/mapflow/_wf_mean1
INFO:workflow:Running: 3dTstat -prefix mean /h

Couldn't import dot_parser, loading of dot files will not be possible.


In [36]:
wf1._write_report_info

<bound method Workflow._write_report_info of wf1>